# 第四章作业

本次作业我们加强对图嵌入模型的实践，具体地，我们需要利用gensim.Word2Vec模型来实现Node2Vec模型。

在开始实践之前，首先请同学回答两个问题：
1. Node2Vec模型中的p参数和q参数各代表什么意思？
2. 在Node2Vec采样随机游走时，如果我们鼓励随机游走访问之前被采样过的节点，我们应该如何调节p或者q？

## 代码填空 - Node2Vec采样随机游走

In [1]:
# 我们导入需要用到的工具包，如果出现ImportError麻烦大家自己安装一下对应的工具包
import numba
import numpy as np
import scipy.sparse as sp
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import normalize
from sklearn.metrics import accuracy_score

/Users/jinwei/anaconda3/envs/gnn/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
def node2vec(adj, embedding_dim=64, walk_length=30, walks_per_node=10,
                  workers=8, window_size=10, num_neg_samples=1, p=4, q=1):
    """
    参数说明
    -------------
    adj : 图的邻接矩阵
    embedding_dim : 图嵌入的维度
    walk_length : 随机游走的长度
    walks_per_node : 每个节点采样多少个随机游走
    workers: word2vec模型使用的线程数量
    window_size: word2vec模型中使用的窗口大小
    num_neg_samples : 负样本的数量
    p: node2vec的p参数
    q: node2vec的q参数
    """
    walks = sample_n2v_random_walks(adj, walk_length, walks_per_node, p=p, q=q) # 利用随机游走提取共现信息
    walks = [list(map(str, walk)) for walk in walks]
    model = Word2Vec(walks, vector_size=embedding_dim, 
                     negative=num_neg_samples, compute_loss=True)   # 映射函数、重构器、目标
    embedding = model.wv.vectors[np.fromiter(map(int, model.wv.index_to_key), np.int32).argsort()] # 从词向量中取出节点嵌入
    return embedding

def sample_n2v_random_walks(adj, walk_length, walks_per_node, p, q):
    """
    返回值的类型
    -------
    walks : np.ndarray, shape [num_walks * num_nodes, walk_length]
        采样后的随机游走
    """
    adj = sp.csr_matrix(adj)
    random_walks = _n2v_random_walk(adj.indptr,
                                    adj.indices,
                                    walk_length,
                                    walks_per_node,
                                    p,
                                    q)
    return random_walks 

In [3]:
### 不用numba加速的版本
def _n2v_random_walk(indptr,
                    indices,
                    walk_length,
                    walks_per_node,
                    p,
                    q):
    N = len(indptr) - 1 # 节点数量
    final_walks = [] # 存储所有的随机游走
    for _ in range(walks_per_node):
        for n in range(N):
            walk = [n]
            ######################################
            #
            #      同学们需要自己完成这部分代码
            #
            #
            ######################################
            final_walks.append(walk)
    return np.array(final_walks)


# ### 用numba加速的版本
# # 建议debug阶段把下面这行注释掉，debug通过后再把取消下面这行的注释
# # @numba.jit(nopython=True)
# def _n2v_random_walk(indptr,
#                     indices,
#                     walk_length,
#                     walks_per_node,
#                     p,
#                     q):
#     N = len(indptr) - 1 # 节点数量

#     for _ in range(walks_per_node):
#         for n in range(N):
#             walk = [n]
#             ######################################
#             #
#             #      同学们需要自己完成这部分代码
#             #
#             #
#             ######################################
#             yield walk # 用yield来构造一个generator

在上面的过程中，大家会需要根据概率来采样数组中的元素，请大家直接使用下面这个`random_choice`函数，它跟`numpy.random.choice`类似，但是这个函数可以支持在numba中使用不同概率来采样。详情可见这个页面<https://github.com/numba/numba/issues/2539#issuecomment-507306369>

In [4]:
@numba.jit(nopython=True)
def random_choice(arr, p):
    """
    参数说明
    ----------
    arr : 1-D 数组
    p : 数组中每个元素对应的概率
    
    返回值
    -------
    samples : 采样后的元素
    """
    return arr[np.searchsorted(np.cumsum(p), np.random.random(), side="right")]

## 代码填空- 完成测试代码

In [5]:
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import to_scipy_sparse_matrix
dataset = Planetoid(root='./data', name='Cora')# 将数据保存在data文件夹下
data = dataset[0]
adj = to_scipy_sparse_matrix(data.edge_index)

In [6]:
embedding = node2vec(adj, embedding_dim=64, p=0.5, q=0.5)
embedding.shape

(2708, 64)

In [7]:
# 请大家完成下面这个测试函数
def evaluate_node_classification(embedding_matrix, labels, train_mask, 
                                 test_mask, normalize_embedding=True, max_iter=1000):
        
    """训练一个线性模型（比如逻辑回归模型）来预测节点的标签
    
    返回值说明
    ----
    preds: 模型预测的标签
    test_acc: 模型预测的准确率
    """
    ######################################
    #
    #      同学们需要自己完成这部分代码
    #
    #
    ######################################   
    return preds, test_acc

In [8]:
preds, test_acc = evaluate_node_classification(embedding, data.y, data.train_mask, data.test_mask)
print('Test Acc: %.4f' % test_acc)

Test Acc: 0.6890


## 拓展问题

请同学们调节p参数和q参数，对比Node2Vec模型的效果和DeepWalk模型(p=q=1)的效果。